In [1]:
from webbot import Browser
import pandas as pd
import re
import time
import random

In [2]:
web = Browser()
web.go_to('https://www.strava.com/login') 
web.type('mkarkalik@gmail.com' , into='email')
web.type('akosamas' , into='password' , id='password') # specific selection
web.click('submit', id = 'login-button') # you are logged in ^_^

In [3]:
regex_watch = r'(?<=device spans8)">(.*)(?=</div><div class="gear spans8)'
regex_shoes = r'(?<=Shoes:<span class="gear-name">)(.*)(?=</span></div></div></div></div></div><footer class="achievements row">)'
regex_distance = r'(?<=inline-stats section"><li><strong>)(.*)(?=<abbr class="unit" title="kilometers">km)'

In [4]:
race_table = pd.read_csv('barcelona_half_2019_raw.csv', encoding = "ISO-8859-1")

In [5]:
watch_list = []
shoes_list = []
distance_list = []
i = 0

In [6]:
for url in race_table['race_url']:
    #time.sleep(random.randint(1,4))
    web.go_to('https://www.strava.com'+url)
    activity = web.get_page_source()
    activity_clear = activity.replace('\n','')
    watch = re.findall(regex_watch, activity_clear)
    shoes = re.findall(regex_shoes, activity_clear)
    distance = re.findall(regex_distance, activity_clear)
    watch_list.append(watch)
    shoes_list.append(shoes)
    distance_list.append(distance)
    i = i+1

KeyboardInterrupt: 

In [7]:
pd.DataFrame(watch_list).to_csv('watch_list.csv')
pd.DataFrame(shoes_list).to_csv('shoes_list.csv')
pd.DataFrame(distance_list).to_csv('distance_list.csv')

In [ ]:
#'Too Many Requests' create condition

In [ ]:
# Different aproach

In [78]:
from requests import Session
from bs4 import BeautifulSoup as bs
 
with Session() as s:
    site = s.get("https://www.strava.com/login")
    bs_content = bs(site.content, "html.parser")
    token1 = bs_content.find("meta", {"name":"csrf-token"})["content"]
    token2 = bs_content.find("input", {"name":"authenticity_token"})["value"]
    login_data = {"email":"mkarkalik@gmail.com", 
                  "password":"akosamas", 
                  "csrf-token":token1,
                  "authenticity_token":token2}
    s.post("https://www.strava.com/login", login_data)
    home_page = s.get("https://www.strava.com/")
    print(bs(home_page.content, "html.parser"))
    #print(token2)

<!DOCTYPE html>

<html class="logged-out responsive feed3p0 old-login" dir="ltr" lang="en-US" xmlns="http://www.w3.org/TR/html5" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<!--
layout website_landing_page
-->
<head>
<meta charset="utf-8"/>
<title>
Strava | Run and Cycling Tracking on the Social Network for Athletes
</title>
<meta content="Designed by athletes, for athletes, Strava’s mobile app and website connect millions of runners and cyclists through the sports they love." name="description"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<meta content="width = device-width, initial-scale = 1, maximum-scale = 2" name="viewport"/>
<link href="https://d3nn82uaxijpm6.cloudfront.net/apple-touch-icon-180x180.png?v=dLlWydWlG8" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://d3nn82uaxijpm6.cloudfront.net/apple-touch-icon-152x152.png?v=dLlWydWlG8" 

In [ ]:
race_table['Watch'] = watch_list
race_table['Shoes'] = shoes_list
race_table['Distance'] = distance_list

In [ ]:
race_table.to_csv('barcelona_half_2019.csv')